# Demo illustrating how Flip can be used in teaching FEM  

## Manual FEM Assembly and Solution Using a Python Beam Library

This notebook demonstrates how to:

- define a simple beam model,
- extract element stiffness matrices,
- assemble the global stiffness matrix manually,
- assemble the global load vector manually,
- apply boundary conditions manually,
- solve the system,
- and compute internal forces per element.

The library is used only to provide element matrices, DOF mappings, and basic model structure — all core FEM steps are done explicitly.


### Imports and basic setup

In [1]:
import sys
sys.path.append("..")

import numpy as np
from flip import Domain, Beam2D, Node, Material, CrossSection


## Define a simple 2‑span beam model

In [2]:
# Create domain
d = Domain()

# Add nodes (3 nodes, 2 elements)
d.add_node(Node(1, d, coords=[0.0, 0.0, 0.0]))
d.add_node(Node(2, d, coords=[4.0, 0.0, 0.0]))
d.add_node(Node(3, d, coords=[8.0, 0.0, 0.0]))

# Material and cross-section
d.add_material(Material("m1", e=210.e9))
d.add_cs(CrossSection("cs1", a=0.02, iy=8e-6))

# Add elements
d.add_element(Beam2D(1, d, nodes=(1, 2), mat="m1", cs="cs1"))
d.add_element(Beam2D(2, d, nodes=(2, 3), mat="m1", cs="cs1"))



## Inspect element stiffness matrices (local)

In [3]:
K_e_local = {}

# Pretty-print settings
np.set_printoptions(
    suppress=True,   # Suppress scientific notation
    precision=4,     # Decimal places
    floatmode='fixed',  # Fixed-point notation
    linewidth=120    # Prevent line breaks
)
for eid, elem in d.elements.items():
    K_e_local[eid] = elem.compute_local_stiffness()['answer']
    print(f"Element {eid} local stiffness matrix K_e:\n{K_e_local[eid]}\n")


Element 1 local stiffness matrix K_e:
[[ 1.0500e+09  0.0000e+00  0.0000e+00 -1.0500e+09  0.0000e+00  0.0000e+00]
 [ 0.0000e+00  3.1500e+05 -6.3000e+05  0.0000e+00 -3.1500e+05 -6.3000e+05]
 [ 0.0000e+00 -6.3000e+05  1.6800e+06  0.0000e+00  6.3000e+05  8.4000e+05]
 [-1.0500e+09  0.0000e+00  0.0000e+00  1.0500e+09  0.0000e+00  0.0000e+00]
 [ 0.0000e+00 -3.1500e+05  6.3000e+05  0.0000e+00  3.1500e+05  6.3000e+05]
 [ 0.0000e+00 -6.3000e+05  8.4000e+05  0.0000e+00  6.3000e+05  1.6800e+06]]

Element 2 local stiffness matrix K_e:
[[ 1.0500e+09  0.0000e+00  0.0000e+00 -1.0500e+09  0.0000e+00  0.0000e+00]
 [ 0.0000e+00  3.1500e+05 -6.3000e+05  0.0000e+00 -3.1500e+05 -6.3000e+05]
 [ 0.0000e+00 -6.3000e+05  1.6800e+06  0.0000e+00  6.3000e+05  8.4000e+05]
 [-1.0500e+09  0.0000e+00  0.0000e+00  1.0500e+09  0.0000e+00  0.0000e+00]
 [ 0.0000e+00 -3.1500e+05  6.3000e+05  0.0000e+00  3.1500e+05  6.3000e+05]
 [ 0.0000e+00 -6.3000e+05  8.4000e+05  0.0000e+00  6.3000e+05  1.6800e+06]]



## Assemble the global stiffness matrix manually

In [4]:
ndofs = 9
K = np.zeros((ndofs, ndofs))
elementDOFs = [[0, 1, 2, 3, 4, 5], 
               [3, 4, 5, 6, 7, 8]]  # for 2 elements


for eid, elem in d.elements.items():
    dofs = elementDOFs[eid-1]  # length 6 for 2D beam
    Ke = K_e_local[eid]

    for i in range(6):
        for j in range(6):
            K[dofs[i], dofs[j]] += Ke[i, j]

print("Global stiffness matrix K:\n", K)


Global stiffness matrix K:
 [[ 1.0500e+09  0.0000e+00  0.0000e+00 -1.0500e+09  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00]
 [ 0.0000e+00  3.1500e+05 -6.3000e+05  0.0000e+00 -3.1500e+05 -6.3000e+05  0.0000e+00  0.0000e+00  0.0000e+00]
 [ 0.0000e+00 -6.3000e+05  1.6800e+06  0.0000e+00  6.3000e+05  8.4000e+05  0.0000e+00  0.0000e+00  0.0000e+00]
 [-1.0500e+09  0.0000e+00  0.0000e+00  2.1000e+09  0.0000e+00  0.0000e+00 -1.0500e+09  0.0000e+00  0.0000e+00]
 [ 0.0000e+00 -3.1500e+05  6.3000e+05  0.0000e+00  6.3000e+05  0.0000e+00  0.0000e+00 -3.1500e+05 -6.3000e+05]
 [ 0.0000e+00 -6.3000e+05  8.4000e+05  0.0000e+00  0.0000e+00  3.3600e+06  0.0000e+00  6.3000e+05  8.4000e+05]
 [ 0.0000e+00  0.0000e+00  0.0000e+00 -1.0500e+09  0.0000e+00  0.0000e+00  1.0500e+09  0.0000e+00  0.0000e+00]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 -3.1500e+05  6.3000e+05  0.0000e+00  3.1500e+05  6.3000e+05]
 [ 0.0000e+00  0.0000e+00  0.0000e+00  0.0000e+00 -6.3000e+05  8.4000e+05  0.0000e+0

In [5]:
## Assemble the global load vector manually
f = np.zeros(ndofs)
f[5]=10.

## Apply boundary conditions manually

In [6]:
fixed_dofs = [0,1,4,7]  # Fixed DOFs: Node 1 (ux, uz), Node 2 (uz), Node 3 (uz)
unknown_dofs = [i for i in range(ndofs) if i not in fixed_dofs]
print("Fixed DOFs:", fixed_dofs)

Kuu = K[np.ix_(unknown_dofs, unknown_dofs)]
fuu = f[np.ix_(unknown_dofs)]


Fixed DOFs: [0, 1, 4, 7]


## Solve the linear system


In [7]:

ru = np.linalg.solve(Kuu, fuu)
r = np.zeros(ndofs)
r[unknown_dofs] = ru
print("Nodal displacement vector r:\n", r)


Nodal displacement vector r:
 [ 0.0000  0.0000 -0.0000  0.0000  0.0000  0.0000  0.0000  0.0000 -0.0000]


## Recover element‑level displacements and internal forces

In [8]:
for eid, elem in d.elements.items():
    dofs = elementDOFs[eid-1]  # length 6 for 2D beam
    u_elem_local = r[dofs]

    print(f"\nElement {eid}:")
    print("  Global DOFs:", dofs)
    print("  Local displacements u_elem_local:", u_elem_local)

    # Example: compute internal forces at element ends 
    F_element_local = K_e_local[eid] @ u_elem_local
    print("  End forces (local):", F_element_local)




Element 1:
  Global DOFs: [0, 1, 2, 3, 4, 5]
  Local displacements u_elem_local: [ 0.0000  0.0000 -0.0000  0.0000  0.0000  0.0000]
  End forces (local): [ 0.0000 -1.2500  0.0000  0.0000  1.2500  5.0000]

Element 2:
  Global DOFs: [3, 4, 5, 6, 7, 8]
  Local displacements u_elem_local: [ 0.0000  0.0000  0.0000  0.0000  0.0000 -0.0000]
  End forces (local): [ 0.0000 -1.2500  5.0000  0.0000  1.2500 -0.0000]
